Criando uma tool de busca de temperatura

In [18]:
import requests
import datetime

from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field


class GetTemperaturaAtualArgs(BaseModel):
    latitude: float = Field(..., description="Latitude do local a ser consultado")
    longitude: float = Field(..., description="Longitude do local a ser consultado")

def get_temperatura_atual(latitude: float, longitude: float):
    URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    response = requests.get(URL, params=params)

    if response.status_code == 200:
        resultado = response.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        
        temp_atual = resultado["hourly"]["temperature_2m"][index_mais_prox]
        
        return temp_atual
    else:
        raise Exception(f"Erro na requisição: {response.status_code} - {response.text}")
    
    
tool_get_temperatura_atual = StructuredTool.from_function(
    func=get_temperatura_atual,
    name="Tool Get Temperatura Atual",
    description="Retorna a temperatura atual em um local",
    args_schema=GetTemperaturaAtualArgs,
    return_direct=True,
)

tool_get_temperatura_atual.run({"latitude": 52.52, "longitude": 13.41})

10.1

Criando um tool de busca no wikipedia

In [30]:
from typing import List
import wikipedia
wikipedia.set_lang("pt")

query = "Isaac Asimov"

class RetornaResumosWikipediaArgs(BaseModel):
    query: str = Field(..., description="Termo a ser buscado")

def busca_wikipedia(query:str) -> List:
    # titulos das paginas sobre a query
    titulos_paginas = wikipedia.search(query)

    # resumos das paginas
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(titulo, auto_suggest=True)
            resumos.append(f"Titulo da pagina: {titulo}\nResumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return 'Busca nao teve retorno'
    else:
        return '\n\n'.join(resumos)

tool_get_resumos_wikipedia = StructuredTool.from_function(
    func=busca_wikipedia,
    name="Tool Retornar resumos da Wikipedia",
    description="Retorna resumos da Wikipedia",
    args_schema=RetornaResumosWikipediaArgs,
    return_direct=True
)

tool_get_resumos_wikipedia.invoke({"query": "futebol"})


'Titulo da pagina: Futebol\nResumo: O futebol, também referido como futebol de campo, futebol de onze e, originalmente, futebol associado (em inglês: association football, football, soccer), é um esporte de equipe jogado entre dois times de 11 jogadores cada um e um árbitro que se ocupa da correta aplicação das normas. É considerado o desporto mais popular do mundo, pois cerca de 270 milhões de pessoas participam das suas várias competições. É jogado num campo retangular gramado, com uma baliza em cada lado do campo. O objetivo do jogo é deslocar uma bola através do campo para colocá-la dentro da baliza adversária, ação que se denomina golo (português europeu) ou gol (português brasileiro). A equipe que marca mais gols ao término da partida é a vencedora.\nO jogo moderno foi criado na Inglaterra com a formação de The Football Association, cujas regras de 1863 são a base do desporto na atualidade. O órgão regente do futebol é a Federação Internacional de Futebol (em francês: Fédération 

Testando a tool

In [51]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Issac'),
    ('user', '{input}')
])

chat = ChatOpenAI()

tools = [
    convert_to_openai_function(busca_wikipedia),
    convert_to_openai_function(get_temperatura_atual)
]
chain = prompt | chat.bind(functions=tools)

In [52]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 74, 'total_tokens': 86, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4a277985-5d16-48ff-8f9f-bd9134042212-0')

In [53]:
chain.invoke({'input': 'Qual a temperatura de Porto Alegre?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-30.0331,"longitude":-51.23}', 'name': 'get_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 81, 'total_tokens': 107, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-61256c5d-a7a9-4231-b8a7-7212becfb9c1-0')

In [54]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'busca_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 80, 'total_tokens': 100, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-748c436d-bbbe-49f5-ab5e-c1d1ab9d64c3-0')

In [55]:
chain.invoke({'input': 'Quem foi Steve Jobs'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Steve Jobs"}', 'name': 'busca_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 77, 'total_tokens': 94, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-3ed0be0c-1b35-4e30-9155-4718455a953d-0')